# Crime Database
In this project I'll build a database to store data related to crimes that occured in Boston

In [3]:
import psycopg2
conn = psycopg2.connect(dbname='dq', user='dq')
cur = conn.cursor()

Create a database called `crime_db` to hold the data

In [4]:
conn.autocommit = True
cur.execute("CREATE DATABASE crime_db")
conn.close()

Create a schema called `crimes` in the `crime_db`

In [5]:
conn = psycopg2.connect(dbname="crime_db", user="dq")
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes")

Read the data in the `boston.csv` file that holds the crime data

In [6]:
import csv
with open("boston.csv") as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

Create a function that returns a set of all the values in a particular column. This is useful for two reasons:
1. To check whether an enumerated datatype might be a good choice for representing a column
2. Computing the maximum length of any text-like column to select the right sizes for `VARCHAR` columns

In [7]:
def get_col_set(file, col_index):
    import csv
    values = set()
    with open(file, 'r') as file:
        next(file)          # remove the header row
        reader = csv.reader(file)
        for row in reader:
            values.add(row[col_index])
    return values

In [8]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


Get the length of the longest description

In [9]:
descriptions = get_col_set("boston.csv", 2)
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))
print(max_len)    

58


Create a table within the `crimes` schema called `boston_crimes`

Find the lengths of all the column sets to see which columns can be enumerated datatypes.

In [10]:
for i in range(len(col_headers)):
    print(len(get_col_set("boston.csv", i)))

298329
219
239
1177
7
18177
18177


In [11]:
print(get_col_set("boston.csv", 4))

{'Wednesday', 'Monday', 'Saturday', 'Friday', 'Sunday', 'Tuesday', 'Thursday'}


In [12]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


Create an enumerated data type for `the day_of_the_week` column

In [13]:
cur.execute("""
    CREATE TYPE day_of_week AS ENUM ('Monday', 'Tuesday', 'Wednesday',
        'Thursday', 'Friday', 'Saturday', 'Sunday');
            """)

Create a table called `boston_crimes` in the `crimes` schema

In [14]:
cur.execute("""
        CREATE TABLE crimes.boston_crimes(
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week day_of_week,
        lat DECIMAL,
        long DECIMAL
        );""")

Check to see if the `boston_crimes` table was created correctly

In [15]:
cur.execute("""
    SELECT * FROM pg_catalog.pg_tables
    WHERE schemaname != 'pg_catalog' AND 
    schemaname != 'information_schema';
            """)
print(cur.fetchall())

[('crimes', 'boston_crimes', 'dq', None, True, False, False)]


Load the data from `boston.csv` into the `boston_crimes` table

In [16]:
with open("boston.csv") as file:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", file)
cur.execute("SELECT * FROM crimes.boston_crimes")
print(len(cur.fetchall()))

298329


Revoke all privileges inherited from the `public group` and on the `public schema`

In [17]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")

In [18]:
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public")

## Creating User Groups
Creating two user groups `readonly` and `readwrite`
- The `readonly` group will only have privileges to perform SELECT queries
- The `readwrite` group will have privileges to perform SELECT, INSERT, DELETE, and UPDATE queries

Creating the `readonly` group

In [25]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("""
        GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;
            """)

ProgrammingError: role "readonly" already exists


Creating the `readwrite` group

In [26]:
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("""
        GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES
        IN SCHEMA crimes TO readwrite;
            """)

ProgrammingError: role "readwrite" already exists


## Creating Users
Creating users and assigning them to the `readonly` and `readwrite` groups

In [28]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1'")
cur.execute("GRANT readonly TO data_analyst")

ProgrammingError: role "data_analyst" already exists


In [29]:
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2'")
cur.execute("GRANT readwrite TO data_scientist")

## Testing
Make sure that everything is configured correctly.

In [33]:
# close the old connection and create a new one for testing
conn.close()

conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()
cur.execute("""
        SELECT grantee, privilege_type
        FROM information_schema.table_privileges
        WHERE grantee = 'readwrite';
            """)
print(cur.fetchall())

[('readwrite', 'INSERT'), ('readwrite', 'SELECT'), ('readwrite', 'UPDATE'), ('readwrite', 'DELETE')]


In [32]:
cur.execute("""
        SELECT grantee, privilege_type
        FROM information_schema.table_privileges
        WHERE grantee = 'readonly';
            """)
print(cur.fetchall())

[('readonly', 'SELECT')]


In [34]:
cur.execute("""
        SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
        WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
            """)
for user in cur:
    print(user)

('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('readonly', False, False, False, False)
('data_scientist', False, False, False, True)
